# High Resolution inference 

In [1]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/pfz/09-videoseal/videoseal-dev


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from videoseal.utils.display import save_vid
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy
from videoseal.data.datasets import VideoDataset
from videoseal.augmentation import Identity, H264

import os
from tqdm import tqdm
import torch
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu" 

/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from tqdm import tqdm
import torch

# Directory containing videos
video_dir = "assets/videos/metamoviegen"

# Example usage
ckpts = {
    # "hidden": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.0_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=hidden/checkpoint.pth',
    "unet": '/private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth'
}

fps = 24 // 1
# frames_per_clip = fps * 3  # 3s
# frame_step = 1

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
for model_name, ckpt in ckpts.items():
    wam = setup_model_from_checkpoint(ckpt)
    wam.eval()
    wam.to(device)

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][1:]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {model_name}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        timer.start()
        vid, mask = VideoDataset.load_full_video_decord(video_path)
        print(f"loading video {video_path} - took {timer.stop():.2f}s")

        # Watermark embedding
        timer.start()
        outputs = wam.embed(vid, is_video=True)
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs = vid  # b c h w
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # save
        timer.start()
        save_vid(imgs, f"{model_name}_{base_name}.mp4", fps)
        save_vid(imgs_w, f"{model_name}_{base_name}_wmed.mp4", fps)
        save_vid(diff, f"{model_name}_{base_name}_wm.mp4", fps)

        # Compute min and max values, reshape, and normalize
        min_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).min(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        max_vals = diff.view(imgs.shape[0], imgs.shape[1], -1).max(dim=2, keepdim=True)[0].view(imgs.shape[0], imgs.shape[1], 1, 1)
        normalized_images = (diff - min_vals) / (max_vals - min_vals)

        # Save the normalized video
        save_vid(normalized_images, f"{model_name}_{base_name}_wm_normalized.mp4", fps)
        print(f"saving videos - took {timer.stop():.2f}s")

        # Augment video
        print(f"compressing and detecting watermarks")
        for crf in [23, 40]:
            imgs_aug, _ = H264()(imgs_w, crf=crf)

            # detect
            timer.start()
            outputs = wam.detect(imgs_aug, is_video=True)
            preds = outputs["preds"]
            bit_preds = preds[:, 1:]  # b k ...
            bit_accuracy_ = bit_accuracy(
                bit_preds,
                msgs
            ).nanmean().item()
            print(f"CRF={crf} Bit Accuracy: {bit_accuracy_:.3f} - detection took {timer.stop():.2f}s")

        del vid, mask, outputs, imgs, imgs_w, diff, min_vals, max_vals, normalized_images

    # Free model from GPU
    del wam
    torch.cuda.empty_cache()

Model loaded successfully from /private/home/hadyelsahar/work/code/videoseal/2024_logs/videoseal0.1/_lambda_d=0.5_lambda_i=0.5_optimizer=AdamW,lr=1e-4_videowam_step_size=4_video_start=500_embedder_model=unet_small2/checkpoint.pth


Processing Videos for unet:   0%|          | 0/3 [00:00<?, ?it/s]

loading video assets/videos/metamoviegen/metamoviegen2.mp4 - took 9.55s
embedding watermark  - took 7.98s
saving videos - took 34.19s
compressing and detecting watermarks
CRF=23 Bit Accuracy: 0.999 - detection took 2.03s
CRF=40 Bit Accuracy: 0.878 - detection took 2.15s


Processing Videos for unet:  33%|███▎      | 1/3 [02:16<04:32, 136.12s/it]

loading video assets/videos/metamoviegen/metamoviegen3.mp4 - took 9.36s
embedding watermark  - took 7.63s
saving videos - took 33.19s
compressing and detecting watermarks
CRF=23 Bit Accuracy: 0.997 - detection took 1.77s
CRF=40 Bit Accuracy: 0.702 - detection took 1.85s


Processing Videos for unet:  67%|██████▋   | 2/3 [04:21<02:09, 129.71s/it]

loading video assets/videos/metamoviegen/metamoviegen4.mp4 - took 9.80s
embedding watermark  - took 7.72s
saving videos - took 32.68s
compressing and detecting watermarks
CRF=23 Bit Accuracy: 0.998 - detection took 1.81s
CRF=40 Bit Accuracy: 0.894 - detection took 1.74s


Processing Videos for unet: 100%|██████████| 3/3 [06:28<00:00, 129.58s/it]
